In [2]:
import re
from pathlib import Path

import duckdb
import pandas as pd
from IPython.display import display, HTML

In [3]:
RESULTS_PATH = Path("misc")
list(RESULTS_PATH.glob("*"))

[PosixPath('misc/results_split_train.csv'),
 PosixPath('misc/results_floodnet_initial_msl.csv'),
 PosixPath('misc/results_floodnet_finetune.csv'),
 PosixPath('misc/results_single_double.csv'),
 PosixPath('misc/results_split_msl.csv'),
 PosixPath('misc/results_split_test.csv')]

In [4]:
(res := pd.read_csv(RESULTS_PATH / "results_single_double.csv").sort_index())


,Name,State,Created,Runtime,epoch,challenge_score_safe,f1_loc_safe,f1_class_safe,f1_safe,f1_0_safe,...,precision_1_safe,precision_2_safe,precision_3_safe,precision_4_safe,recall_safe,recall_0_safe,recall_1_safe,recall_2_safe,recall_3_safe,recall_4_safe
0,t_farseg_single,finished,2024-10-30T23:47:31.000Z,92,0,0.654576,0.768670,0.605679,0.662679,0.986025,...,0.741384,0.478886,0.620301,0.654507,0.637581,0.986403,0.780518,0.344296,0.556610,0.520078
1,t_baseline,finished,2024-10-30T23:36:56.000Z,91,0,0.717108,0.815655,0.674874,0.717499,0.989278,...,0.816330,0.480730,0.618351,0.774877,0.702057,0.991759,0.732577,0.449922,0.653548,0.682479
2,t_dahitra_custom_weights,finished,2024-10-30T19:48:02.000Z,105,0,0.713039,0.814833,0.669413,0.712418,0.989117,...,0.804639,0.480846,0.629536,0.754783,0.695439,0.991027,0.750752,0.444770,0.633370,0.657273
3,dahitra_custom_weights-2024-10-30T04:59:24-bpa...,finished,2024-10-30T03:59:26.000Z,53898,39,0.706949,0.814514,0.660850,0.711308,0.995109,...,0.807438,0.402872,0.638461,0.747992,0.706753,0.995742,0.753322,0.482893,0.589710,0.712095
4,t_farseg-double,finished,2024-10-29T23:50:02.000Z,87,0,0.746176,0.856936,0.698707,0.739565,0.991671,...,0.854739,0.515673,0.657176,0.830408,0.714702,0.994107,0.784694,0.454124,0.674212,0.666372
5,t_farseg_single,finished,2024-10-29T23:43:58.000Z,85,0,0.654576,0.768670,0.605679,0.662679,0.986025,...,0.741384,0.478886,0.620301,0.654507,0.637581,0.986403,0.780518,0.344296,0.556610,0.520078
6,t_baseline,finished,2024-10-29T23:38:15.000Z,85,0,0.717108,0.815655,0.674874,0.717499,0.989278,...,0.816330,0.480730,0.618351,0.774877,0.702057,0.991759,0.732577,0.449922,0.653548,0.682479
7,t_dahitra,finished,2024-10-29T23:35:09.000Z,85,0,0.713686,0.818459,0.668783,0.713558,0.989118,...,0.788521,0.469305,0.630613,0.762014,0.701089,0.989933,0.771928,0.441310,0.643220,0.659055
8,t_baseline_singlebranch,finished,2024-10-29T23:27:45.000Z,89,0,0.632918,0.716731,0.596998,0.646710,0.982451,...,0.680635,0.411071,0.562457,0.579772,0.650097,0.981402,0.711425,0.417133,0.568919,0.571606
9,baseline_singlebranch,crashed,2024-10-29T06:30:03.000Z,59266,39,0.630278,0.720151,0.591760,0.643973,0.992481,...,0.687674,0.415151,0.567375,0.614363,0.634058,0.992313,0.711840,0.411282,0.514728,0.540124


In [5]:
def attach_index(df, name_pattern):
    name = df["Name"]
    match = name.str.extract(name_pattern)
    match.columns = ["pass", "model", "branches"]
    match["branches"] = match.apply(lambda row: "double" if row["branches"] == "" else "single", axis=1)
    match["pass"] = match.apply(lambda row: "train" if row["pass"] == "" else "test", axis=1)
    idx = pd.MultiIndex.from_frame(match)
    return df.set_index(idx)

pattern = r"^((?:t_)|(?:))((?:farseg|baseline|dahitra))_?((?:single)|(?:)).*?$"
_df = attach_index(res, pattern).sort_index()
_df = _df[~_df.index.duplicated(keep='first')]
_df

Name  \
pass  model    branches                                                      
test  baseline double                                           t_baseline   
               single                              t_baseline_singlebranch   
      dahitra  double                             t_dahitra_custom_weights   
      farseg   double                                      t_farseg-double   
               single                                      t_farseg_single   
train baseline double                baseline-2024-10-25T14:23:00-bpanzhzq   
               single                                baseline_singlebranch   
      dahitra  double    dahitra_custom_weights-2024-10-30T04:59:24-bpa...   
      farseg   double           farseg-double-2024-10-24T08:34:28-bpanzhzq   
               single           farseg_single-2024-10-25T00:48:08-bpanzhzq   

                            State                   Created  Runtime  epoch  \
pass  model    branches                                                       
test  baseline double    finished  2024-10-30T23:36:56.000Z       91      0   
               single    finished  2024-10-29T23:27:45.000Z       89      0   
      dahitra  double    finished  2024-10-30T19:48:02.000Z      105      0   
      farseg   double    finished  2024-10-29T23:50:02.000Z       87      0   
               single    finished  2024-10-30T23:47:31.000Z       92      0   
train baseline double    finished  2024-10-25T12:23:01.000Z    73826     39   
               single     crashed  2024-10-29T06:30:03.000Z    59266     39   
      dahitra  double    finished  2024-10-30T03:59:26.000Z    53898     39   
      farseg   double    finished  2024-10-24T06:34:29.000Z    49554     39   
               single    finished  2024-10-24T22:48:10.000Z    41633     39   

                         challenge_score_safe  f1_loc_safe  f1_class_safe  \
pass  model    branches                                                     
test  baseline double                0.717108     0.815655       0.674874   
               single                0.632918     0.716731       0.596998   
      dahitra  double                0.713039     0.814833       0.669413   
      farseg   double                0.746176     0.856936       0.698707   
               single                0.654576     0.768670       0.605679   
train baseline double                0.708839     0.816051       0.662891   
               single                0.630278     0.720151       0.591760   
      dahitra  double                0.706949     0.814514       0.660850   
      farseg   double                0.756942     0.861971       0.711930   
               single                0.635641     0.769079       0.578454   

                          f1_safe  f1_0_safe  ...  precision_1_safe  \
pass  model    branches                       ...                     
test  baseline double    0.717499   0.989278  ...          0.816330   
               single    0.646710   0.982451  ...          0.680635   
      dahitra  double    0.712418   0.989117  ...          0.804639   
      farseg   double    0.739565   0.991671  ...          0.854739   
               single    0.662679   0.986025  ...          0.741384   
train baseline double    0.712736   0.995135  ...          0.800129   
               single    0.643973   0.992481  ...          0.687674   
      dahitra  double    0.711308   0.995109  ...          0.807438   
      farseg   double    0.749631   0.996385  ...          0.863679   
               single    0.640966   0.994147  ...          0.747986   

                         precision_2_safe  precision_3_safe  precision_4_safe  \
pass  model    branches                                                         
test  baseline double            0.480730          0.618351          0.774877   
               single            0.411071          0.562457          0.579772   
      dahitra  double            0.480846          0.629536          0.754783   
 

In [6]:
df = _df[["Name"]+ [f"{col}_safe" for col in [
    "challenge_score", "f1_loc", "f1_class"
] + [f"f1_{i}" for i in range(5)]]]
df.columns = [col[:-5] if col.endswith("_safe") else col for col in df.columns]
df

df1 = pd.concat([
    df.loc["train"].add_prefix("train_"),
    df.loc["test"].add_prefix("test_")
], axis=1)
df1

train_Name  \
model    branches                                                      
baseline double                baseline-2024-10-25T14:23:00-bpanzhzq   
         single                                baseline_singlebranch   
dahitra  double    dahitra_custom_weights-2024-10-30T04:59:24-bpa...   
farseg   double           farseg-double-2024-10-24T08:34:28-bpanzhzq   
         single           farseg_single-2024-10-25T00:48:08-bpanzhzq   

                   train_challenge_score  train_f1_loc  train_f1_class  \
model    branches                                                        
baseline double                 0.708839      0.816051        0.662891   
         single                 0.630278      0.720151        0.591760   
dahitra  double                 0.706949      0.814514        0.660850   
farseg   double                 0.756942      0.861971        0.711930   
         single                 0.635641      0.769079        0.578454   

                   train_f1_0  train_f1_1  train_f1_2  train_f1_3  train_f1_4  \
model    branches                                                               
baseline double      0.995135    0.778230    0.438287    0.631754    0.720275   
         single      0.992481    0.699548    0.413207    0.539771    0.574857   
dahitra  double      0.995109    0.779442    0.439268    0.613118    0.729602   
farseg   double      0.996385    0.832977    0.504691    0.667324    0.746778   
         single      0.994147    0.765162    0.400260    0.502363    0.542900   

                                  test_Name  test_challenge_score  \
model    branches                                                   
baseline double                  t_baseline              0.717108   
         single     t_baseline_singlebranch              0.632918   
dahitra  double    t_dahitra_custom_weights              0.713039   
farseg   double             t_farseg-double              0.746176   
         single             t_farseg_single              0.654576   

                   test_f1_loc  test_f1_class  test_f1_0  test_f1_1  \
model    branches                                                     
baseline double       0.815655       0.674874   0.989278   0.772189   
         single       0.716731       0.596998   0.982451   0.695690   
dahitra  double       0.814833       0.669413   0.989117   0.776762   
farseg   double       0.856936       0.698707   0.991671   0.818220   
         single       0.768670       0.605679   0.986025   0.760448   

                   test_f1_2  test_f1_3  test_f1_4  
model    branches                                   
baseline double     0.464816   0.635462   0.725749  
         single     0.414080   0.565670   0.575660  
dahitra  double     0.462105   0.631447   0.702661  
farseg   double     0.482945   0.665585   0.739402  
         single     0.400588   0.586733   0.579600

In [7]:
df2 = df1.reset_index()
query = f"""
SELECT
    model, branches,
    train_challenge_score "train_score",
    test_challenge_score "test_score",
    test_challenge_score - train_challenge_score "test_change_abs",
    (test_challenge_score - train_challenge_score) * 100 / train_challenge_score "test_change_percent"
FROM df2
WHERE model NOT LIKE 'dahitra'
ORDER BY model, branches DESC
"""
display(HTML(duckdb.query(query).df().to_html()))
for i in range(5):
    query = f"""
    SELECT
        model, branches,
        train_f1_{i},
        test_f1_{i},
        test_f1_{i} - train_f1_{i} "test_f1_{i}_change_abs",
        (test_f1_{i} - train_f1_{i}) * 100 / train_f1_{i} "test_f1_{i}_change_percent"
    FROM df2
    WHERE model NOT LIKE 'dahitra'
    ORDER BY model, branches DESC
    """
    print(f"################## CLASS {i} ##################")
    display(HTML(duckdb.query(query).df().to_html()))

,model,branches,train_score,test_score,test_change_abs,test_change_percent
0,baseline,single,0.630278,0.632918,0.002640,0.418940
1,baseline,double,0.708839,0.717108,0.008269,1.166565
2,farseg,single,0.635641,0.654576,0.018935,2.978875
3,farseg,double,0.756942,0.746176,-0.010767,-1.422392


################## CLASS 0 ##################


,model,branches,train_f1_0,test_f1_0,test_f1_0_change_abs,test_f1_0_change_percent
0,baseline,single,0.992481,0.982451,-0.010030,-1.010578
1,baseline,double,0.995135,0.989278,-0.005857,-0.588545
2,farseg,single,0.994147,0.986025,-0.008122,-0.817008
3,farseg,double,0.996385,0.991671,-0.004714,-0.473106


################## CLASS 1 ##################


,model,branches,train_f1_1,test_f1_1,test_f1_1_change_abs,test_f1_1_change_percent
0,baseline,single,0.699548,0.695690,-0.003859,-0.551605
1,baseline,double,0.778230,0.772189,-0.006041,-0.776210
2,farseg,single,0.765162,0.760448,-0.004714,-0.616072
3,farseg,double,0.832977,0.818220,-0.014756,-1.771531


...

,model,branches,train_f1_2,test_f1_2,test_f1_2_change_abs,test_f1_2_change_percent
0,baseline,single,0.413207,0.414080,0.000873,0.211173
1,baseline,double,0.438287,0.464816,0.026529,6.052764
2,farseg,single,0.400260,0.400588,0.000329,0.082104
3,farseg,double,0.504691,0.482945,-0.021746,-4.308690


################## CLASS 3 ##################


,model,branches,train_f1_3,test_f1_3,test_f1_3_change_abs,test_f1_3_change_percent
0,baseline,single,0.539771,0.565670,0.025899,4.798148
1,baseline,double,0.631754,0.635462,0.003708,0.587004
2,farseg,single,0.502363,0.586733,0.084370,16.794635
3,farseg,double,0.667324,0.665585,-0.001739,-0.260552


################## CLASS 4 ##################


,model,branches,train_f1_4,test_f1_4,test_f1_4_change_abs,test_f1_4_change_percent
0,baseline,single,0.574857,0.575660,0.000803,0.139738
1,baseline,double,0.720275,0.725749,0.005474,0.759950
2,farseg,single,0.542900,0.579600,0.036700,6.759982
3,farseg,double,0.746778,0.739402,-0.007376,-0.987736


In [8]:
df3 = pd.concat([
    df.loc["test", :, "single"].add_prefix("single_"),
    df.loc["test", :, "double"].add_prefix("double_"),
], axis=1)
df3

,single_Name,single_challenge_score,single_f1_loc,single_f1_class,single_f1_0,single_f1_1,single_f1_2,single_f1_3,single_f1_4,double_Name,double_challenge_score,double_f1_loc,double_f1_class,double_f1_0,double_f1_1,double_f1_2,double_f1_3,double_f1_4
model,,,,,,,,,,,,,,,,,,
baseline,t_baseline_singlebranch,0.632918,0.716731,0.596998,0.982451,0.695690,0.414080,0.565670,0.57566,t_baseline,0.717108,0.815655,0.674874,0.989278,0.772189,0.464816,0.635462,0.725749
farseg,t_farseg_single,0.654576,0.768670,0.605679,0.986025,0.760448,0.400588,0.586733,0.57960,t_farseg-double,0.746176,0.856936,0.698707,0.991671,0.818220,0.482945,0.665585,0.739402
dahitra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,t_dahitra_custom_weights,0.713039,0.814833,0.669413,0.989117,0.776762,0.462105,0.631447,0.702661


In [50]:

df4 = df3.reset_index()
query = f"""
SELECT
    model,
    double_challenge_score "double_score",
    single_challenge_score "single_score",
    single_challenge_score - double_challenge_score "score_change_abs",
    (single_challenge_score - double_challenge_score) * 100 / single_challenge_score "score_change_percent",
    double_f1_loc "double_score",
    single_f1_loc "single_score",
    single_f1_loc - double_f1_loc "f1_loc_change_abs",
    (single_f1_loc - double_f1_loc) * 100 / single_f1_loc "f1_loc_change_percent",
FROM df4
WHERE model NOT LIKE 'dahitra'
ORDER BY model
"""
res_global = duckdb.query(query).df()
display(HTML(res_global.to_html()))

class_res = []
for i in range(5):
    query = f"""
    SELECT
        model,
        double_f1_{i},
        single_f1_{i},
        single_f1_{i} - double_f1_{i} "single_f1_{i}_change_abs",
        (single_f1_{i} - double_f1_{i}) * 100 / double_f1_{i} "single_f1_{i}_change_percent"
    FROM df4
    WHERE model NOT LIKE 'dahitra'
    ORDER BY model
    """
    class_res.append(duckdb.query(query).df())
    print(f"################## CLASS {i} ##################")
    display(HTML(class_res[i].to_html()))

,model,double_score,single_score,score_change_abs,score_change_percent,double_score_1,single_score_1,f1_loc_change_abs,f1_loc_change_percent
0,baseline,0.717108,0.632918,-0.084190,-13.301899,0.815655,0.716731,-0.098924,-13.802109
1,farseg,0.746176,0.654576,-0.091599,-13.993673,0.856936,0.768670,-0.088265,-11.482872


################## CLASS 0 ##################


,model,double_f1_0,single_f1_0,single_f1_0_change_abs,single_f1_0_change_percent
0,baseline,0.989278,0.982451,-0.006827,-0.690081
1,farseg,0.991671,0.986025,-0.005646,-0.569353


################## CLASS 1 ##################


,model,double_f1_1,single_f1_1,single_f1_1_change_abs,single_f1_1_change_percent
0,baseline,0.772189,0.695690,-0.076499,-9.906817
1,farseg,0.818220,0.760448,-0.057772,-7.060717


...

,model,double_f1_2,single_f1_2,single_f1_2_change_abs,single_f1_2_change_percent
0,baseline,0.464816,0.414080,-0.050736,-10.915315
1,farseg,0.482945,0.400588,-0.082357,-17.053069


################## CLASS 3 ##################


,model,double_f1_3,single_f1_3,single_f1_3_change_abs,single_f1_3_change_percent
0,baseline,0.635462,0.565670,-0.069793,-10.982995
1,farseg,0.665585,0.586733,-0.078853,-11.847117


################## CLASS 4 ##################


,model,double_f1_4,single_f1_4,single_f1_4_change_abs,single_f1_4_change_percent
0,baseline,0.725749,0.57566,-0.150089,-20.680556
1,farseg,0.739402,0.57960,-0.159802,-21.612287


No i jakie niby są wnioski z tego

- Duży spadek na klasie 4 przez zupełnie zniszczone budynki, model poradzi sobie z przypisaniem poprawnej klasy ale nie będzie umiał ładnie obrysować budynku, bo nie ma dostępu do zdjęcia przed
- Można macierze błędów zestawić
- No to lecimy z gradcamem

In [49]:
print(res_global.to_latex(
    index=False,
    decimal=",",
    float_format="%.3f",
    header=[
        "Model",
        "\\textit{{Score}} (2)",
        "\\textit{{Score}} (1)",
        "Różnica",
        r"Różnica (\%)",
        "F1_{{LOC}} (2)",
        "F1_{{LOC}} (1)",
        "Różnica",
        r"Różnica (\%)"
    ]
))

\begin{tabular}{lrrrrrrrr}
\toprule
Model & \textit{Score} (2) & \textit{Score} (1) & Różnica & Różnica (\%) & F1_{LOC} (2) & F1_{LOC} (1) & Różnica & Różnica (\%) \\
\midrule
baseline & 0,717 & 0,633 & -0,084 & -13,302 & 0,816 & 0,717 & -0,099 & -13,802 \\
farseg & 0,746 & 0,655 & -0,092 & -13,994 & 0,857 & 0,769 & -0,088 & -11,483 \\
\bottomrule
\end{tabular}



\begin{tabular}{lrrrrrrrr}
\toprule
Model & \textit{Score} (2) & \textit{Score} (1) & Różnica & F1_{LOC} (2) & F1_{LOC} (1) & Różnica\\
\midrule
baseline & 0,717 & 0,633 & -0,084 (-13,3)\% & 0,816 & 0,717 & -0,099 (-13,8\%) \\
farseg & 0,746 & 0,655 & -0,092 & (-13,9)\% & 0,857 & 0,769 & -0,088 (-11,4\%) \\
\bottomrule
\end{tabular}

In [57]:
print(pd.concat([d.rename(columns={k: v for k, v in zip(d.columns, ["model", "f12", "f11", "change_abs", "change_rel"])}) for d in class_res], axis=0).to_latex(
    index=False,
    decimal=",",
    float_format="%.3f",
    header=[
        "Model",
        "$F1_c$ (2)",
        "$F1_c$ (1)",
        "Różnica",
        r"Różnica (\%)",
    ]
))

\begin{tabular}{lrrrr}
\toprule
Model & $F1_c$ (2) & $F1_c$ (1) & Różnica & Różnica (\%) \\
\midrule
baseline & 0,989 & 0,982 & -0,007 & -0,690 \\
farseg & 0,992 & 0,986 & -0,006 & -0,569 \\
baseline & 0,772 & 0,696 & -0,076 & -9,907 \\
farseg & 0,818 & 0,760 & -0,058 & -7,061 \\
baseline & 0,465 & 0,414 & -0,051 & -10,915 \\
farseg & 0,483 & 0,401 & -0,082 & -17,053 \\
baseline & 0,635 & 0,566 & -0,070 & -10,983 \\
farseg & 0,666 & 0,587 & -0,079 & -11,847 \\
baseline & 0,726 & 0,576 & -0,150 & -20,681 \\
farseg & 0,739 & 0,580 & -0,160 & -21,612 \\
\bottomrule
\end{tabular}

